## Try this Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truefoundry/mlfoundry-examples/blob/main/examples/sklearn/ca_housing_regression.ipynb)

## Install dependencies

In [ ]:
! pip install --quiet "numpy>=1.0.0,<2.0.0" "pandas>=1.0.0,<2.0.0" scikit-learn shap==0.40.0
! pip install -U mlfoundry

## Initialize MLFoundry Client

In [ ]:
import os
import getpass
import urllib.parse
import mlfoundry as mlf

In [ ]:
TFY_URL = os.environ.get('TFY_URL', 'https://app.truefoundry.com/')
TFY_API_KEY = os.environ.get('TFY_API_KEY')
if not TFY_API_KEY:
    print(f'Paste your TrueFoundry API key\nYou can find it over at {urllib.parse.urljoin(TFY_URL, "settings")}')
    TFY_API_KEY = getpass.getpass()

In [ ]:
client = mlf.get_client(api_key=TFY_API_KEY)

---

## California Housing Price Prediction as a Regression problem

In [ ]:
import shap
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Load the California Housing dataset

In [ ]:
data = datasets.fetch_california_housing(as_frame=True)
print(data.keys())

In [ ]:
print(data.DESCR)

In [ ]:
data.frame.head()

### Start a MLFoundry Run

In [ ]:
run = client.create_run(project_name='sklearn-ca-housing-example')
print('RUN ID:', run.run_id)
print(f'You can track your runs live at {urllib.parse.urljoin(TFY_URL, "mlfoundry")}')

### Set tags for our run

In [ ]:
run.set_tags({'framework': 'sklearn', 'task': 'regression'})

### Log the dataset

In [ ]:
# Create a Pandas dataframe with all the features
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

run.log_dataset(
    dataset_name='california_housing_dataset',
    features=X,
    actuals=y
)

### Split Dataset into Training and Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
feature_columns = X_train.columns.tolist()
X_train = X_train[feature_columns]
X_test = X_test[feature_columns]

print('Feature columns:', feature_columns)
print('Train samples:', len(X_train))
print('Test samples:', len(X_test))

### Training Model

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=100)
rf_reg.fit(X_train, y_train)

### Logging Parameters & Model

In [ ]:
run.log_params({"n_estimators": 100})
run.log_params(rf_reg.get_params())
run.log_model(rf_reg, framework=mlf.ModelFramework.SKLEARN)

### Logging metrics

In [ ]:
y_pred_train = rf_reg.predict(X_train)
y_pred_test = rf_reg.predict(X_test)

metrics_dict = {
    'train/mae': mean_absolute_error(y_true=y_train, y_pred=y_pred_train),
    'train/mse': mean_squared_error(y_true=y_train, y_pred=y_pred_train),
    'train/r2_score': r2_score(y_true=y_train, y_pred=y_pred_train),
    'test/mae': mean_absolute_error(y_true=y_test, y_pred=y_pred_test),
    'test/mse': mean_squared_error(y_true=y_test, y_pred=y_pred_test),
    'test/r2_score': r2_score(y_true=y_test, y_pred=y_pred_test)
}
print(metrics_dict)
run.log_metrics(metrics_dict)

### Logging Dataset Stats

In [ ]:
X_test_df = X_test.copy()
X_test_df['targets'] = y_test
X_test_df['predictions'] = y_pred_test

# shap value computation
explainer = shap.TreeExplainer(rf_reg)
shap_values = explainer.shap_values(X_test)

run.log_dataset_stats(
    X_test_df, 
    data_slice='test',
    data_schema=mlf.Schema(
        feature_column_names=feature_columns,
        prediction_column_name='predictions',
        actual_column_name='targets'
    ),
    model_type='regression',
    shap_values=shap_values
)

### Optionally, you can also log training data slice as follows:
It might take longer as training split is relatively much larger than test split

```python
X_train_df = X_train.copy()
X_train_df['targets'] = y_train
X_train_df['predictions'] = y_pred_train

# shap value computation
explainer = shap.TreeExplainer(rf_reg)
shap_values = explainer.shap_values(X_train)

run.log_dataset_stats(
    X_train_df, 
    data_slice='train',
    data_schema=mlf.Schema(
        feature_column_names=feature_columns,
        prediction_column_name='predictions',
        actual_column_name='targets'
    ),
    model_type='regression',
    shap_values=shap_values
)
```

In [ ]:
run.end()